#### LeetCode - SQL - #585

Refer [585. Investments in 2016](https://leetcode.com/problems/investments-in-2016/description/)

Write a solution to report the sum of all total investment values in 2016  `tiv_2016`, for all policyholders who:

-   have the same  `tiv_2015`  value as one or more other policyholders, and
-   are not located in the same city as any other policyholder (i.e., the (`lat, lon`) attribute pairs must be unique).

Round  `tiv_2016`  to  **two decimal places**.

In [ ]:
data_insurance = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
columns_insurance = ['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']
schema_insurance = {'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'}

In [ ]:
import pandas

pandas_df_insurance = pandas.DataFrame(data=data_insurance, columns=columns_insurance).astype(schema_insurance)

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.master("local[1]").appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
df_insurance = spark_context.createDataFrame(pandas_df_insurance)

#### Using Window Fuctions

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

window_spec_tiv_2015 = Window.partitionBy(df_insurance.tiv_2015)
window_spec_lat_lan = Window.partitionBy(df_insurance.lat, df_insurance.lon)

df_insurance_summary \
    = df_insurance \
        .withColumn("tiv_2015_count", F.count("*").over(window_spec_tiv_2015)) \
        .withColumn("lat_lan_count", F.count("*").over(window_spec_lat_lan))
df_insurance_summary.show()

In [ ]:
df_insurance_summary \
    .filter((df_insurance_summary.tiv_2015_count > 1) & (df_insurance_summary.lat_lan_count == 1)) \
    .select(F.round(F.sum(df_insurance_summary.tiv_2016), 2).alias("tiv_2016")) \
    .show()

#### Using IN clause

In [ ]:
df_insurance_tiv_2015_multiple \
    = df_insurance.groupBy(df_insurance.tiv_2015).count() \
        .filter(F.col("count") > 1) \
        .select(F.col("tiv_2015"))

# Convert to Python list
list_insurance_tiv_2015_multiple = list(df_insurance_tiv_2015_multiple.toPandas()['tiv_2015'])
print(list_insurance_tiv_2015_multiple)

In [ ]:
df_insurance_lat_lon_unique \
    = df_insurance.groupBy(df_insurance.lat, df_insurance.lon).count() \
        .filter(F.col("count") == 1) \
        .select(F.concat_ws("-", df_insurance.lat, df_insurance.lon).alias("lat_lan"))

# Convert to Python list
list_insurance_lat_lon_unique = list(df_insurance_lat_lon_unique.toPandas()['lat_lan'])
print(list_insurance_lat_lon_unique)

In [ ]:
df_insurance.show()

df_insurance \
    .filter(df_insurance.tiv_2015.isin(list_insurance_tiv_2015_multiple) & F.concat_ws("-", df_insurance.lat, df_insurance.lon).isin(list_insurance_lat_lon_unique)) \
    .select(F.sum(df_insurance.tiv_2016).alias("tiv_2016")) \
    .show()